# Exercise: Build A Spam Classifier Using A Foundation Model

A foundation model serves as a fundamental building block for potentially endless applications. One application we will explore in this exercise is the development of a spam classifier using only prompting. By leveraging the capabilities of a foundation model, this project aims to accurately identify and filter out unwanted and potentially harmful messages.

## Steps

1. Identify and gather relevant data
2. Build and evaluate the spam classifier
3. Build an improved classifier?

In [ ]:
# Student Task: Set up the OpenAI API key and base URL from environment variables
# TODO: If using Vocareum, set the API key directly in the code below

import litellm
import os

if os.getenv("OPENAI_API_KEY"):
    litellm.openai_key = os.getenv("OPENAI_API_KEY")

# If using Vocareum, you can also set your API key here directly
# Uncomment and replace the string with your actual Vocareum API key
# litellm.openai_key = "voc-**********"

if (litellm.openai_key or "").startswith("voc-"):
    litellm.api_base = "https://openai.vocareum.com/v1"
    print("Using Vocareum OpenAI API base URL")

: 

## Step 1: Identify and gather relevant data

To train and test the spam classifier, you will need a dataset of messages that are labeled as spam or not spam. It is important to identify and gather a suitable dataset that represents a wide range of spam and non-spam messages.

In [2]:
# Load the sms_spam dataset using the datasets library
# No changes needed in this cell

from datasets import load_dataset

dataset = load_dataset("sms_spam", split=["train"])[0]

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label = entry["label"]
    print(f"label={label}, sms={sms}")

label=0, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=0, sms=Ok lar... Joking wif u oni...

label=1, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



Those labels could be easier to read. Let's create some functions to convert numerical ids to labels.

In [3]:
# Let's map the numeric labels to human-readable labels
# No changes needed in this cell

id2label = {0: "NOT SPAM", 1: "SPAM"}
label2id = {"NOT SPAM": 0, "SPAM": 1}

for entry in dataset.select(range(3)):
    sms = entry["sms"]
    label_id = entry["label"]
    print(f"label={id2label[label_id]}, sms={sms}")

label=NOT SPAM, sms=Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

label=NOT SPAM, sms=Ok lar... Joking wif u oni...

label=SPAM, sms=Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's



## Step 2: Build and evaluate the spam email classifier

Using the foundation model and the prepared dataset, you can create a spam email classifier.

Let's write a prompt that will ask the model to classify 15 message as either "spam" or "not spam". For easier parsing, we can ask the LLM to respond in JSON.

In [4]:
# Create a helper function to get multiple sms messages as a single string
# No changes needed in this cell


def get_sms_messages_string(dataset, item_numbers):
    sms_messages_string = ""
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]
        label_id = entry["label"]

        sms_messages_string += f"{item_number} -> {sms}\n"

    return sms_messages_string


print(get_sms_messages_string(dataset, [0, 1, 2]))

0 -> Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...

1 -> Ok lar... Joking wif u oni...

2 -> Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's




Now let's write a bit of code that will produce your prompt. Your prompt should include a few SMS message to be labelled as well as instructions for the LLM.

Some LLMs will also format the output for you as JSON if you ask them, e.g. "Respond in JSON format."

In [5]:
# Student task: Construct a query that includes `sms_messages_string` and asks the LLM
# to classify the messages as SPAM or NOT SPAM, responding in JSON format.
# TODO: Fill in the missing parts marked with **********

# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# The input should be of the form
# 11 -> ...
# 16 -> ...
# 23 -> ...

# The output should be of the form
# {
#     "11": "NOT SPAM",
#     "16": "SPAM",
#     "23": "NOT SPAM"
# }

SYSTEM_PROMPT = """**********"""
USER_PROMPT = sms_messages_string

# <<< START SOLUTION SECTION

SYSTEM_PROMPT = """You are a helpful assistant that classifies text messages as SPAM or NOT SPAM.

    You will receive a list of text messages, each with a unique identifier.

    Respond only with JSON. Do not include any other text, preamble, or explanation.

    EXAMPLE INPUT:
    11 -> ...
    16 -> ...
    23 -> ...

    EXAMPLE OUTPUT:
    {
        "11": "NOT SPAM",
        "16": "SPAM",
        "23": "NOT SPAM"
    }
    """

# >>> END SOLUTION SECTION

print("SYSTEM PROMPT:")
print(SYSTEM_PROMPT)
print("\nUSER PROMPT:")
print(USER_PROMPT)


SYSTEM PROMPT:
You are a helpful assistant that classifies text messages as SPAM or NOT SPAM.

    You will receive a list of text messages, each with a unique identifier.

    Respond only with JSON. Do not include any other text, preamble, or explanation.

    EXAMPLE INPUT:
    11 -> ...
    16 -> ...
    23 -> ...

    EXAMPLE OUTPUT:
    {
        "11": "NOT SPAM",
        "16": "SPAM",
        "23": "NOT SPAM"
    }
    

USER PROMPT:
7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune

8 -> WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.

9 -> Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030

10 -> I'm gonna be home soon and i don't want to talk about thi

In [6]:
# Use litellm to get the response
# No changes needed in this cell

from litellm import completion

response = completion(
    model="gpt-5-nano",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
response = response.choices[0].message.content
print(response)

# Check that response is in valid JSON format
try:
    import json

    json.loads(response)
    print("Response is valid JSON")
except json.JSONDecodeError:
    print("Response is not valid JSON")


{
  "7": "SPAM",
  "8": "SPAM",
  "9": "SPAM",
  "10": "NOT SPAM",
  "11": "SPAM",
  "12": "SPAM",
  "13": "NOT SPAM",
  "14": "NOT SPAM"
}
Response is valid JSON


In [7]:
# Write a function that estimates the accuracy of your classifier
# by comparing your responses to the labels in the dataset
# No changes needed in this cell


def get_accuracy(response, dataset, original_indices):
    correct = 0
    total = 0

    if isinstance(response, str):
        import json

        try:
            response = json.loads(response)
        except json.JSONDecodeError as e:
            print("Error decoding JSON response:", e)
            return

    for entry_number, prediction in response.items():
        if int(entry_number) not in original_indices:
            continue

        label_id = dataset[int(entry_number)]["label"]
        label = id2label[label_id]

        # If the prediction from the LLM matches the label in the dataset
        # we increment the number of correct predictions.
        # (Since LLMs do not always produce the same output, we use the
        # lower case version of the strings for comparison)
        if prediction.lower() == label.lower():
            correct += 1
        else:
            print(
                f"Mismatch for entry {entry_number}: predicted={prediction}, actual={label}"
            )

        # increment the total number of predictions
        total += 1

    try:
        accuracy = correct / total
    except ZeroDivisionError:
        print("No matching results found!")
        return

    return round(accuracy, 2)


print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15))}")

Mismatch for entry 7: predicted=SPAM, actual=NOT SPAM
Accuracy: 0.88


That's not bad! (Or if it got them all, even better!)

It might not be correct for every example we throw at it, but it's a great start, especially for not giving it any examples or training data.

We can see that the model is able to distinguish between spam and non-spam messages with a high degree of accuracy. This is a great example of how a foundation model can be used to build a spam email classifier.

## Step 3: Build an improved classifier?

If you provide the LLM with some examples for how to complete a task, it will sometimes improve its performance. Let's try that out here.

In [8]:
# Create a helper function to get a few-shot examples string
# No changes needed in this cell


def get_few_shot_examples_string(dataset, item_numbers):
    examples_string = ""

    examples_string += "EXAMPLE INPUT:\n"
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        sms = entry["sms"]

        examples_string += f"{item_number} -> {sms}\n"

    examples_string += "EXAMPLE OUTPUT:\n"
    examples_string += "{\n"
    for item_number, entry in zip(item_numbers, dataset.select(item_numbers)):
        label_id = entry["label"]
        label = id2label[label_id]
        examples_string += f'    "{item_number}": "{label}",\n'
    examples_string += "}\n"

    return examples_string


print(get_few_shot_examples_string(dataset, range(54, 60)))

EXAMPLE INPUT:
54 -> SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV

55 -> Do you know what Mallika Sherawat did yesterday? Find out now @  &lt;URL&gt;

56 -> Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! 

57 -> Sorry, I'll call later in meeting.

58 -> Tell where you reached

59 -> Yes..gauti and sehwag out of odi series.

EXAMPLE OUTPUT:
{
    "54": "SPAM",
    "55": "NOT SPAM",
    "56": "SPAM",
    "57": "NOT SPAM",
    "58": "NOT SPAM",
    "59": "NOT SPAM",
}



In [9]:
# Student task: Construct a query that includes the labelled messages
# and asks the LLM to classify the unlabelled messages as SPAM or NOT SPAM,
# responding in JSON format.
# TODO: Fill in the missing parts marked with **********

# Get a few messages and format them as a string
sms_messages_string = get_sms_messages_string(dataset, range(7, 15))

# Construct a SYSTEM_PROMPT and USER_PROMPT to send to the LLM. This time, let's include the
# labelled examples to see if it improves accuracy. Hint: use get_few_shot_examples_string(dataset, range(54, 60))

SYSTEM_PROMPT = """**********"""
USER_PROMPT = """**********"""


# <<< START SOLUTION SECTION

SYSTEM_PROMPT = f"""You are a helpful assistant that classifies text messages as SPAM or NOT SPAM.

    You will receive a list of text messages, each with a unique identifier.

    Respond only with JSON. Do not include any other text, preamble, or explanation.

{get_few_shot_examples_string(dataset, range(54, 60))}
    """
USER_PROMPT = sms_messages_string

# >>> END SOLUTION SECTION

print("SYSTEM_PROMPT:")
print(SYSTEM_PROMPT)
print("USER_PROMPT:")
print(USER_PROMPT)

SYSTEM_PROMPT:
You are a helpful assistant that classifies text messages as SPAM or NOT SPAM.

    You will receive a list of text messages, each with a unique identifier.

    Respond only with JSON. Do not include any other text, preamble, or explanation.

EXAMPLE INPUT:
54 -> SMS. ac Sptv: The New Jersey Devils and the Detroit Red Wings play Ice Hockey. Correct or Incorrect? End? Reply END SPTV

55 -> Do you know what Mallika Sherawat did yesterday? Find out now @  &lt;URL&gt;

56 -> Congrats! 1 year special cinema pass for 2 is yours. call 09061209465 now! C Suprman V, Matrix3, StarWars3, etc all 4 FREE! bx420-ip4-5we. 150pm. Dont miss out! 

57 -> Sorry, I'll call later in meeting.

58 -> Tell where you reached

59 -> Yes..gauti and sehwag out of odi series.

EXAMPLE OUTPUT:
{
    "54": "SPAM",
    "55": "NOT SPAM",
    "56": "SPAM",
    "57": "NOT SPAM",
    "58": "NOT SPAM",
    "59": "NOT SPAM",
}

    
USER_PROMPT:
7 -> As per your request 'Melle Melle (Oru Minnaminunginte Nur

Paste in your response from the LLM below:

In [10]:
# Get the response from the LLM
# No changes needed in this cell

from litellm import completion

response = completion(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": USER_PROMPT},
        {"role": "system", "content": SYSTEM_PROMPT},
    ],
)

response = response.choices[0].message.content
print(response)

# Check that response is in valid JSON format
try:
    import json

    json.loads(response)
    print("Response is valid JSON")
except json.JSONDecodeError:
    print("Response is not valid JSON")

{
  "7": "SPAM",
  "8": "SPAM",
  "9": "SPAM",
  "10": "NOT SPAM",
  "11": "SPAM",
  "12": "SPAM",
  "13": "NOT SPAM",
  "14": "NOT SPAM"
}
Response is valid JSON


Let's check the accuracy now

In [11]:
# Estimate the accuracy of your classifier by comparing your responses to the labels in the dataset
# No changes needed in this cell

print(f"Accuracy: {get_accuracy(response, dataset, range(7, 15)):.2f}")

Mismatch for entry 7: predicted=SPAM, actual=NOT SPAM
Accuracy: 0.88


Do you still have any errors present? If so, why do you think this is happening?

<br /><br /><br /><br /><br /><br /><br /><br /><br />